# ODIN

In [ ]:
import tensorflow as tf
import tensorflow

from tensorflow import keras
from keras.layers import Dense

import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tqdm import trange

import keras.backend as K
import cv2
from odin import norm_perturbations
from scipy.special import softmax


from PIL import Image


**Loading the pretrained DenseNet model :**

In [ ]:
print ('Loading Network...')
model_path = '../Models/my_model_densenet.h5'
densenet_model  = tf.keras.models.load_model(model_path)
print(' Done.')

Loading Network...


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Colocations handled automatically by placer.


 Done.


# Image generator

In [ ]:
from argument_parser import myParser
args = myParser()

Training using the following parameters:
augment: True
batch_size: 40
checkpoint_frequency: 1000
dataset_root: /usr/xtmp/hannah/segkeras/nn-isic2019/dataset/
decay_start_iteration: 15000
detailed_logs: False
early_stop_patience: 8
experiment_root: FL_Aug_MEL
gpu_device: 0
heavy_augment: False
init_epoch: 0
initial_checkpoint: None
is_train: 1
learning_rate: 1e-06
load_weights: None
loading_threads: 8
loss_type: 1
margin: soft
num_classes: 7
num_epochs: 25
num_gpus: 1
resume: False
sizeH: 224
sizeW: 224
skip_class: MEL
train_iterations: 25000


In [ ]:
#Fitz Darker only
from data_generators import fitzpatrick17kImageDataset
## Loading Data Generators
ROOT = '/fitzpatrick 17k /'

print('Loading Data Generators...')

args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2 # Validation Set
Testing_generator_FST_V-VI = fitzpatrick17kImageDataset(args)

print('   Done.')


Loading Data Generators...
Loading Testing Dataset...
   Done.


In [ ]:
#Fitz Lighter only
from data_generators import fitzpatrick17kImageDataset
## Loading Data Generators
ROOT = '/fitzpatrick 17k /'

print('Loading Data Generators...')

args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2 # Validation Set  change the condition on common.py before running
Testing_generator_FST_I-IV = fitzpatrick17kImageDataset(args)

print('   Done.')



Loading Data Generators...
Loading Testing Dataset...
   Done.


In [ ]:
#Fitz ALL images
from data_generators import fitzpatrick17kImageDataset
## Loading Data Generators
ROOT = '/fitzpatrick 17k /'

print('Loading Data Generators...')

args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 0 # Validation Set
Testing_generator_ALL = fitzpatrick17kImageDataset(args)

print('   Done.')



Loading Data Generators...
Loading Validation Dataset...
   Done.


In [ ]:
#ISIC
from data_generators import ISICImageDataset
## Loading Data Generators
ROOT = '/ISIC_2019_Training_Input/'

print('Loading Data Generators...')
args.is_train = 2 # Training Set
args.num_classes = 6
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2 # Validation Set
Validation_generator = ISICImageDataset(args)
print('   Done.')

Loading Data Generators...
class_names:['MEL', 'NV', 'BCC', 'AK', 'BKL', 'SCC']
skip_class:['DF', 'VASC']
Loading Testing Dataset...
   Done.


In [ ]:
def compute_baseline_softmax_scores(model, generator_in, generator_out):
    """
    Calculate the base confidence of the output, no perturbation added here, no temperature scaling used.
    Directly copy the original prediction results.
    """
    print('Begin to compute baseline softmax scores')
    distributions = ['In', 'Out']


    for dist in distributions:
        if dist == 'In':
            generator = generator_in
        elif dist == 'Out':
            generator = generator_out

        with open("./densenet121_Base_{}.txt".format(dist), 'w') as f:
            for i in range(len(generator.fids)):
                # print('thisFid: '+str(generator.fids[i]))
                thisImg = Image.open(generator.fids[i]+'.jpg').convert("RGB")
                thisImg = np.expand_dims(np.asarray(thisImg.resize((224,224)))/255, axis =0)
                softmax_probs = model.predict(thisImg)
                softmax_score = np.max(softmax_probs)
                f.write("{}\n".format(softmax_score))
        f.close()

In [ ]:
def get_perturbation_helper_func(model, temperature, num_classes):
    """ Return Keras functions for calculating perturbations. """
    # Compute loss based on the second last layer's output and temperature scaling
    dense_pred_layer_output = model.get_layer('dense_4').output
    scaled_dense_pred_output = dense_pred_layer_output / temperature

    print(dense_pred_layer_output)
    print(scaled_dense_pred_output)

    print(K.argmax(model.outputs))

    label_tensor = K.one_hot(K.argmax(model.outputs), 1)
    tf.print(label_tensor)
    loss = K.sparse_categorical_crossentropy(label_tensor, scaled_dense_pred_output, from_logits=True)
    grad_loss = K.gradients(loss, model.inputs)

    compute_perturbations = K.function(model.inputs + [K.learning_phase()], grad_loss)
    get_scaled_dense_pred_output = K.function(model.inputs + [K.learning_phase()], [scaled_dense_pred_output])

    return compute_perturbations, get_scaled_dense_pred_output

# Find the best parameters :

In [ ]:
def compute_odin_softmax_scores(model, generator_in, generator_out, num_classes = 6, batch_size = 1):
    distributions = ['In', 'Out']

    # This file is used for recording what parameter combinations were already computed.
    progress_file = os.path.join('Done.txt')
    done_set = set()
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            done_set = set(line.rstrip('\n') for line in f)

    # ODIN parameters

    model_name = 'DenseNet201'
    #Grid Search
    temperatures = [1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]

    magnitudes = np.round(np.arange(0, 0.0041, 0.0002), 4)

    need_norm_perturbations = 1



#     model_param_map = get_transfer_model_param_map()
    image_data_format = K.image_data_format()
    learning_phase = 0 # 0 = test, 1 = train



    for temperature in temperatures:
        compute_perturbations, get_scaled_dense_pred_output = get_perturbation_helper_func(model, temperature, num_classes)

        for magnitude in magnitudes:
            for dist in distributions:
                # Skip if the parameter combination has done
                param_comb_id = "{}, {}, {}".format( dist, temperature, magnitude)
                if param_comb_id in done_set:
                    print('Skip ', param_comb_id)
                    continue

                if dist == 'In':
                    generator = generator_in
                elif dist == 'Out':
                    generator = generator_out

                print("\n===== Temperature: {}, Magnitude: {}, {}-Distribution =====".format(temperature, magnitude, dist))

                try:
                    f = open("./densenet121_ODIN_{}_{}_{}.txt".format(temperature, magnitude,dist), 'w')
                    for i in trange(len(generator.fids)):
                        thisFid = generator.fids[i]
                        images = np.expand_dims(cv2.resize(np.array(Image.open(thisFid+'.jpg').convert("RGB"))/255, (224,224), interpolation=cv2.INTER_LINEAR), axis = 0)

                        perturbations = compute_perturbations([images, learning_phase])[0]
                    # Get sign of perturbations
                        perturbations = np.sign(perturbations)

                    # Normalize the perturbations to the same space of image
                    # https://github.com/facebookresearch/odin/issues/5
                    # Perturbations divided by ISIC Training Set STD
                        if need_norm_perturbations:
                            perturbations = norm_perturbations(perturbations, image_data_format)

                    # Add perturbations to images
                        perturbative_images = images - magnitude * perturbations

                    # Calculate the confidence after adding perturbations
                        dense_pred_outputs = get_scaled_dense_pred_output([perturbative_images, learning_phase])[0]
                        softmax_probs = softmax(dense_pred_outputs)
                        softmax_scores = np.max(softmax_probs, axis=-1)

                        for s in softmax_scores:
                            f.write("{}\n".format(s))
                    f.close()

                    with open(progress_file, 'a') as f_done:
                        f_done.write("{}\n".format(param_comb_id))

                except FileNotFoundError:
                    # Handle the "File not found" error
                    print(f"File not found for {param_comb_id}. Skipping...")
                    continue
                last_processed_combination = param_comb_id



In [ ]:
from odin import norm_perturbations
from scipy.special import softmax

def compute_odin_softmax_scores(model, generator_in, generator_out, num_classes=6, batch_size=1):
    distributions = ['In', 'Out']

    # This file is used for recording what parameter combinations were already computed.
    progress_file = os.path.join('Done.txt')
    done_set = set()
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            done_set = set(line.rstrip('\n') for line in f)

    # ODIN parameters
    model_name = 'DenseNet201'
    temperatures = [1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
    #temperatures 200

    magnitudes = np.round(np.arange(0.0002, 0.0041, 0.0002), 4)
    #magnitudes = np.round([0.0002, 0.0041], 4)
    need_norm_perturbations = 1
    image_data_format = K.image_data_format()
    learning_phase = 0  # 0 = test, 1 = train

    for temperature in temperatures:
        compute_perturbations, get_scaled_dense_pred_output = get_perturbation_helper_func(model, temperature, num_classes)

        for magnitude in magnitudes:
            for dist in distributions:
                # Skip if the parameter combination has done
                param_comb_id = "{}, {}, {}".format(dist, temperature, magnitude)
                if param_comb_id in done_set:
                    print('Skip ', param_comb_id)
                    continue

                if dist == 'In':
                    generator = generator_in
                elif dist == 'Out':
                    generator = generator_out

                print("\n===== Temperature: {}, Magnitude: {}, {}-Distribution =====".format(temperature, magnitude, dist))

                # Check if the output file already exists, and if it does, skip processing
                output_file_path = "./densenet121_ODIN_{}_{}_{}.txt".format(temperature, magnitude, dist)
                if os.path.exists(output_file_path):
                    print(f"Output file {output_file_path} already exists. Skipping...")
                    continue

                try:
                    f = open(output_file_path, 'w')
                    for i in trange(len(generator.fids)):
                        thisFid = generator.fids[i]
                        images = np.expand_dims(cv2.resize(np.array(Image.open(thisFid + '.jpg').convert("RGB")) / 255,
                                                           (224, 224), interpolation=cv2.INTER_LINEAR), axis=0)

                        perturbations = compute_perturbations([images, learning_phase])[0]
                        perturbations = np.sign(perturbations)

                        if need_norm_perturbations:
                            perturbations = norm_perturbations(perturbations, image_data_format)

                        perturbative_images = images - magnitude * perturbations

                        dense_pred_outputs = get_scaled_dense_pred_output([perturbative_images, learning_phase])[0]
                        softmax_probs = softmax(dense_pred_outputs)
                        softmax_scores = np.max(softmax_probs, axis=-1)

                        for s in softmax_scores:
                            f.write("{}\n".format(s))
                    f.close()

                    with open(progress_file, 'a') as f_done:
                        f_done.write("{}\n".format(param_comb_id))

                except Exception as e:
                    # Handle any exceptions that might occur during file processing
                    print(f"An error occurred for {param_comb_id}: {str(e)}")
                    continue

                last_processed_combination = param_comb_id


In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_V-VI)

In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_I-IV)

In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_ALL)

In [ ]:
#testing the subset of parameters


need_norm_perturbations = 1
best_auroc = 0.0
best_auroc_params = None
lowest_fpr = float('inf')
lowest_fpr_params = None


#We want to loop over all the saved scores
In_scores_files ="/Models/ODIN/Saved scores/In"
Out_scores_files = "/Models/ODIN/Saved scores/Out"




in_dist_file="densenet121_ODIN_{}_{}_In.txt".format(temperature, magnitude)
out_dist_file="densenet121_ODIN_{}_{}_Out.txt".format(temperature, magnitude)



 # Calculate AUROC
auroc_value = auroc(in_dist_file=  , out_dist_file=out_dist_file)

                    # Calculate FPR
tpr, fpr = get_tpr_and_fpr(scores_in_test, scores_out_test, delta)

                    # Update best AUROC and FPR if necessary
if auroc_value > best_auroc:
   best_auroc = auroc_value
   best_auroc_params = (temperature, magnitude, dist)

if fpr < lowest_fpr:
   lowest_fpr = fpr
   lowest_fpr_params = (temperature, magnitude, dist)

print("Best AUROC: {:.4f}, Parameters: Temperature={}, Magnitude={}, Distribution={}"
      .format(best_auroc, *best_auroc_params))

    # Print the lowest FPR and corresponding parameters
print("Lowest FPR: {:.4f}, Parameters: Temperature={}, Magnitude={}, Distribution={}"
      .format(lowest_fpr, *lowest_fpr_params))

In [ ]:
compute_odin_softmax_scores(densenet_model,Validation_generator, Testing_generator_FST_V-VI)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax:0", shape=(1, ?), dtype=int64)
Skip  In, 1000, 0.0

===== Temperature: 1000, Magnitude: 0.0, Out-Distribution =====


100%|██████████| 1318/1318 [27:31<00:00,  1.25s/it]



===== Temperature: 1000, Magnitude: 0.0002, In-Distribution =====


100%|██████████| 2495/2495 [29:22<00:00,  1.42it/s]



===== Temperature: 1000, Magnitude: 0.0002, Out-Distribution =====


...

100%|██████████| 2495/2495 [30:12<00:00,  1.38it/s]



===== Temperature: 1000, Magnitude: 0.0014, Out-Distribution =====


100%|██████████| 1318/1318 [15:34<00:00,  1.41it/s]



===== Temperature: 1000, Magnitude: 0.0016, In-Distribution =====


 41%|████      | 1013/2495 [12:01<18:12,  1.36it/s]

In [ ]:
compute_odin_softmax_scores(densenet_model,Validation_generator, Testing_generator_FST_I-IV)

In [ ]:
from scipy.special import softmax
from tqdm import trange

# ODIN parameters
#temperatures = [1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
#magnitudes = np.round(np.arange(0, 0.0041, 0.0002), 4)
optimal_temperature = 200
optimal_magnitude = 0.0002
optimal_delta = 0.90385


def compute_odin_softmax_scores (model, generator_in, generator_out, num_classes=6, batch_size=1):
    distributions = ['In', 'Out']

    # Specify the desired starting magnitude and temperature
    start_magnitude = 0.0008
    start_temperature = 1000

    # Find the indices for the desired magnitude and temperature
    start_magnitude_idx = np.where(magnitudes == start_magnitude)[0][0]
    start_temperature_idx = temperatures.index(start_temperature)

    # This file is used for recording what parameter combinations were already computed.
    progress_file = os.path.join('Done.txt')
    done_set = set()
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            done_set = set(line.rstrip('\n') for line in f)

    image_data_format = K.image_data_format()
    learning_phase = 0  # 0 = test, 1 = train

    need_norm_perturbations = 1

    for temperature_idx, temperature in enumerate(temperatures[start_temperature_idx:]):

        if temperature_idx == 0:
            magnitude_start_idx = start_magnitude_idx
        else:
            magnitude_start_idx = 0

        compute_perturbations, get_scaled_dense_pred_output = get_perturbation_helper_func(model, temperature, num_classes)

        for magnitude_idx, magnitude in enumerate(magnitudes[magnitude_start_idx:]):
            for dist in distributions:
                # Skip if the parameter combination has been done
                param_comb_id = "{}, {}, {}".format(dist, temperature, magnitude)
                if param_comb_id in done_set:
                    print('Skip ', param_comb_id)
                    continue

                if dist == 'In':
                    generator = generator_in
                elif dist == 'Out':
                    generator = generator_out

                print("\n===== Temperature: {}, Magnitude: {}, {}-Distribution =====".format(temperature, magnitude, dist))

                f = open("./densenet121_ODIN_{}_{}_{}.txt".format(temperature, magnitude, dist), 'w')

                # Start from the beginning
                for i in trange(len(generator.fids)):
                    try:
                        thisFid = generator.fids[i]
                        images = np.expand_dims(
                            cv2.resize(np.array(Image.open(thisFid + '.jpg').convert("RGB")) / 255, (224, 224),
                                       interpolation=cv2.INTER_LINEAR), axis=0)

                        perturbations = compute_perturbations([images, learning_phase])[0]
                        # Get the sign of perturbations
                        perturbations = np.sign(perturbations)

                        # Normalize the perturbations to the same space of the image
                        if need_norm_perturbations:
                            perturbations = norm_perturbations(perturbations, image_data_format)

                        # Add perturbations to images
                        perturbative_images = images - magnitude * perturbations

                        # Calculate the confidence after adding perturbations
                        dense_pred_outputs = get_scaled_dense_pred_output([perturbative_images, learning_phase])[0]
                        softmax_probs = softmax(dense_pred_outputs)
                        softmax_scores = np.max(softmax_probs, axis=-1)
                        for s in softmax_scores:
                            f.write("{}\n".format(s))
                    except FileNotFoundError:
                        print(f"File not found for index {i}. Continuing...")
                        continue

                f.close()

                with open(progress_file, 'a') as f_done:
                    f_done.write("{}\n".format(param_comb_id))


## Setting the optimal parameters :

In [ ]:
import os
import numpy as np
import keras.backend as K
import cv2
from PIL import Image
from odin import norm_perturbations
from odin import get_tpr_and_fpr, auroc
from scipy.special import softmax
from tqdm import trange
from sklearn.metrics import roc_curve, auc

# Optimal ODIN Parameters
optimal_temperature = 200
optimal_magnitude = 0.0002
optimal_delta = 0.90385

def compute_odin_parameters_and_metrics(model, generator_in, generator_out, num_classes=6, batch_size=1):
    distributions = ['In', 'Out']
    compute_perturbations, get_scaled_dense_pred_output = get_perturbation_helper_func(model, optimal_temperature, num_classes)

    # This file is used for recording what parameter combinations were already computed.
    progress_file = os.path.join('Done.txt')
    done_set = set()
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            done_set = set(line.rstrip('\n') for line in f)

    image_data_format = K.image_data_format()
    learning_phase = 0  # 0 = test, 1 = train

    need_norm_perturbations = 1

    # Initialize variables to store metrics
    auroc_max = -1  # Initialize AUROC to a very low value
    fpr_min = 1     # Initialize FPR to a very high value
    odinparam_auroc_max = None
    odinparam_fpr_min = None

    for dist in distributions:
        for magnitude in [optimal_magnitude]:
            # Skip if the parameter combination has been done
            param_comb_id = "{}, {}, {}".format(dist, optimal_temperature, magnitude)
            if param_comb_id in done_set:
                print('Skip ', param_comb_id)
                continue

            if dist == 'In':
                generator = generator_in
            elif dist == 'Out':
                generator = generator_out

            print("\n===== Temperature: {}, Magnitude: {}, {}-Distribution =====".format(optimal_temperature, magnitude, dist))

            f = open("./densenet121_ODIN_{}_{}_{}.txt".format(optimal_temperature, magnitude, dist), 'w')

            # Start from the beginning
            for i in trange(len(generator.fids)):
                try:
                    thisFid = generator.fids[i]
                    images = np.expand_dims(
                        cv2.resize(np.array(Image.open(thisFid + '.jpg').convert("RGB")) / 255, (224, 224),
                                   interpolation=cv2.INTER_LINEAR), axis=0)

                    perturbations = compute_perturbations([images, learning_phase])[0]
                    # Get the sign of perturbations
                    perturbations = np.sign(perturbations)

                    # Normalize the perturbations to the same space of the image
                    if need_norm_perturbations:
                        perturbations = norm_perturbations(perturbations, image_data_format)

                    # Add perturbations to images
                    perturbative_images = images - magnitude * perturbations

                    # Calculate the confidence after adding perturbations
                    dense_pred_outputs = get_scaled_dense_pred_output([perturbative_images, learning_phase])[0]
                    softmax_probs = softmax(dense_pred_outputs)
                    softmax_scores = np.max(softmax_probs, axis=-1)
                    for s in softmax_scores:
                        f.write("{}\n".format(s))
                except FileNotFoundError:
                    print(f"File not found for index {i}. Continuing...")
                    continue

            f.close()

            with open(progress_file, 'a') as f_done:
                f_done.write("{}\n".format(param_comb_id))


# Call the function to compute ODIN parameters and metrics
#compute_odin_parameters_and_metrics(densenet_model, Validation_generator, Testing_generator_lighter_only)


In [ ]:
from scipy.special import softmax
from tqdm import trange

# Fixed ODIN Parameters
#optimal_temperature = 2
#optimal_magnitude = 0.0002

optimal_temperature = 200
optimal_magnitude = 0.0002

def compute_odin_softmax_scores_fixed(model, generator_in, generator_out, num_classes=6, batch_size=1):
    distributions = ['Out']
    compute_perturbations, get_scaled_dense_pred_output = get_perturbation_helper_func(model, optimal_temperature, num_classes)

    # This file is used for recording what parameter combinations were already computed.
    progress_file = os.path.join('Done.txt')
    done_set = set()
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            done_set = set(line.rstrip('\n') for line in f)

    image_data_format = K.image_data_format()
    learning_phase = 0  # 0 = test, 1 = train
    need_norm_perturbations = 1

    for dist in distributions:
        param_comb_id = "{}, {}, {}".format(dist, optimal_temperature, optimal_magnitude)
        if param_comb_id in done_set:
            print('Skip ', param_comb_id)
            continue

        if dist == 'In':
            generator = generator_in
        elif dist == 'Out':
            generator = generator_out

        print("\n===== Temperature: {}, Magnitude: {}, {}-Distribution =====".format(optimal_temperature, optimal_magnitude, dist))

        f = open("./densenet121_ODIN_{}_{}_{}.txt".format(optimal_temperature, optimal_magnitude, dist), 'w')

        for i in trange(len(generator.fids)):
            try:
                thisFid = generator.fids[i]
                images = np.expand_dims(
                    cv2.resize(np.array(Image.open(thisFid + '.jpg').convert("RGB")) / 255, (224, 224),
                               interpolation=cv2.INTER_LINEAR), axis=0)

                perturbations = compute_perturbations([images, learning_phase])[0]
                # Get sign of perturbations
                perturbations = np.sign(perturbations)

                # Normalize the perturbations to the same space of image
                if need_norm_perturbations:
                    perturbations = norm_perturbations(perturbations, image_data_format)

                # Add perturbations to images
                perturbative_images = images - optimal_magnitude * perturbations

                # Calculate the confidence after adding perturbations
                dense_pred_outputs = get_scaled_dense_pred_output([perturbative_images, learning_phase])[0]
                softmax_probs = softmax(dense_pred_outputs)
                softmax_scores = np.max(softmax_probs, axis=-1)
                for s in softmax_scores:
                    f.write("{}\n".format(s))
            except FileNotFoundError:
                print(f"File not found for index {i}. Continuing...")
                continue

        f.close()

        with open(progress_file, 'a') as f_done:
            f_done.write("{}\n".format(param_comb_id))


## ODIN scores

In [ ]:
compute_odin_softmax_scores_fixed(densenet_model,Validation_generator, Testing_generator_FST_V-VI)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv_6:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax_12:0", shape=(1, ?), dtype=int64)

===== Temperature: 200, Magnitude: 0.0002, Out-Distribution =====


100%|██████████| 1318/1318 [15:24<00:00,  1.43it/s]


In [ ]:
compute_odin_softmax_scores_fixed(densenet_model,Validation_generator, Testing_generator_FST_I-IV)

In [ ]:
compute_odin_softmax_scores_fixed(densenet_model, Validation_generator, Testing_generator_ALL)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv_2:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax_4:0", shape=(1, ?), dtype=int64)

===== Temperature: 200, Magnitude: 0.0002, Out-Distribution =====


100%|██████████| 12222/12222 [2:11:44<00:00,  1.55it/s]


## Baseline scores

In [ ]:
#lighter only
compute_baseline_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_I-IV)

Begin to compute baseline softmax scores


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
#darker only
compute_baseline_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_V-VI)

Begin to compute baseline softmax scores


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
#ALL
compute_baseline_softmax_scores(densenet_model, Validation_generator, Testing_generator_ALL)

Begin to compute baseline softmax scores


In [ ]:
#find the ptimal threshold:
# Concatenate the scores and create corresponding labels

scores_in = np.loadtxt("/content/densenet121_ODIN_200_0.0002_In_Darker.txt")
scores_out = np.loadtxt("/content/densenet121_ODIN_200_0.0002_Out_Darker.txt")
scores_D = np.concatenate([scores_in, scores_out])
labels_D = np.concatenate([np.zeros(len(scores_in)), np.ones(len(scores_out))])

    # Use ROC curve to find the optimal threshold
fpr, tpr, thresholds = roc_curve(labels_D, scores_D)
optimal_threshold_D = thresholds[np.argmax(tpr - fpr)]

In [ ]:
print(optimal_threshold) #ODIN

0.17385952174663544


In [ ]:
print(optimal_threshold) #NN Softmax

0.9962658286094666
